In [401]:
import numpy as np
import sympy as sy
import pylab as py 
from random import random

In [390]:
def stationary_distribution(T):
    """
    Given a random matrix T, returns a stationary sol a, s.t. Ta=a
    
    INPUT:
        - T = Some square matrix.
        
    OUTPUT:
        - A column vector. The stationary vector of the matrix.   
    """
    
    d=len(T)
    A=np.copy(T)
    I=np.identity(d)
    A=A-I
    A=np.transpose(A)
    i=np.ones(d)
    A=np.vstack([A,i])
    A=np.array(A)
    b=np.append([0 for i in range(d)],[[1]])
    return (np.linalg.lstsq(A,b))[0]

In [9]:
def decay_timescale(T):
    """
    It computes the decay or relaxation timescale of an ergodic Markov chain. In such a chain,
    the amplitudes of the distortions from equilibrium decay exponentially with discrete time k.
    
    INPUT:
        -
        
    OUTPUT:
        - 
    """
    
    eig=np.linalg.eig(T)
    eig=eig[0]
    eig=np.absolute(eig)
    t=np.ones(len(eig))
    return np.divide(t,(-1*(np.log(eig))))

In [421]:
def mean_recurrence_time(T):
    """
    If the stationary distribution π exists then this means that the "long-term" probability of being in 
    state i is given by π(i). One can think of these probabilities as the proportion 
    of time on average that the system spends in states. So the mean recurrence times are given by 1/π(i).
    
    INPUT:
        - T: An ergodic markov transition matrix.
        
    OUTPUT:
        - 
    """
    
    pi=stationary_distribution(T)
    mr=np.ones(len(pi))
    return np.divide(mr,pi)

In [11]:
def speed_of_convergence_k(T,p0,k):
    """
    It gives an approximation on the expected speed of convergence of an ergodic Markov chain. 
    It can be bounded by an exponential decay with the slowest timescale t2 in the system. For times much
    greater than k >> t3, all but the first term vanish and we get an approximation: 
    E(k)=(eig[2]))^k * norm(<p0,r2>*l2) where <p0,r2> is the scalar product.
    
    INPUT:
        - T:
        - p0:
        - k:
        
    OUTPUT:
        -
    """
    
    eig=np.linalg.eig(T)
    eig_val=eig[0]
    eig_val= np.sort(eig_val)
    
    right_eig=eig[1]
    left_eig=np.linalg.eig(np.transpose(T))
    left_eig=left_eig[1]
    
    print((eig_val[len(eig_val)-2]))
    print(np.power((eig_val[len(eig_val)-2]),k))
    E_k = (np.power((eig_val[len(eig_val)-2]),k))*(np.linalg.norm((p0*right_eig)*left_eig))
    return E_k

In [205]:
def normalize(M):
    """
    Subfunction for T. It normalizes the matrix given as input.
    
    INPUT:
        - M = A matrix M.
        
    OUTPUT:
        - M0 = The matrix M normalized, with rows that add to 1.     
    """
    
    M0=np.array(M)
    if M0.ndim == 1:
        s= np.absolute(M0).sum()
        return np.divide(M0,s)
        
    elif M0.ndim == 2:
        s=M0.sum(axis=1)
        return np.divide(M0,s[:,np.newaxis])
    else:
        return "Normalize. Wrong input"
    
#----------------------------------------------#

def stationary_sol(T):
    """
    Given a random matrix T, returns a stationary sol a, s.t. aT=a
    
    INPUT:
        - T = Some square matrix.
        
    OUTPUT:
        - A column vector. The stationary vector of the matrix.   
    """
    
    T0=normalize(T)
    T0=np.transpose(T)
    eig=np.linalg.eig(T0)
    for i in range(len(eig[0])):
        if np.around(eig[0][i])==1: #considers only integer part
            pi=np.absolute(np.real(eig[1][:,i]))
            return np.transpose(np.matrix(pi))
    return "No eigenvalue 1"

In [383]:
print(P)
pi=stationary_distribution(P)
print(pi)
c=np.matrix(pi)
c=np.transpose(c)
print(c)
P*c

[[ 0.25  0.2 ]
 [ 0.75  0.8 ]]
[ 0.21052632  0.78947368]
[[ 0.21052632]
 [ 0.78947368]]


matrix([[ 0.21052632],
        [ 0.78947368]])

In [209]:
pi=stationary_sol(P)
print(pi)
pi*P

[[ 0.25766265  0.96623494]]


matrix([[ 0.25766265,  0.96623494]])

In [391]:
T=np.matrix([[0.9,0.1,0,0],[0.1,0.895,0.005,0],[0,0.005,0.795,0.2],[0,0,0.2,0.8]])
stationary_distribution(T)

array([ 0.25,  0.25,  0.25,  0.25])

In [15]:
p0=np.array([1,1,1,1])
speed_of_convergence_k(T,p0,10)

0.990376027604
0.907823029267


1.1122307841811363

In [16]:
speed_of_convergence_k(T,p0,100)

0.990376027604
0.380199862473


0.46580663582088266

In [17]:
speed_of_convergence_k(T,p0,500)

0.990376027604
0.00794437575199


0.0097331517130661762

In [19]:
mean_recurrence_time(T)

array([ 162.25951512,   53.15490883,    2.02495952,    2.07819386])

In [290]:
def paso(i,T):
    """
    Function that simulates a random pass from state i in a markov model. It gives back the vertex
    that is reached.
    
    INPUT:
        - i: the original vertex.
        - T: the Markov transition matrix.
        
    OUTPUT:
        - j: the vertex reached from i in this random experiment.
    """

    T=np.squeeze(np.asarray(T))
    Fi=[sum(T[i][:j+1]) for j in range(len(T[i]))]
    a=random()
    k=0
    while a>=Fi[k]:
        k+=1
    return k

In [72]:
"Test to check whether paso works."

m=1000

vertex=[0,1,2,3]
L=[[],[],[],[]]

for i in range(m):
    for v in vertex:
        L[v].append(paso(v,T))
for v in vertex:
    print((L[v].count(0))/m,(L[v].count(1))/m,(L[v].count(2))/m,(L[v].count(3))/m)

0.893 0.107 0.0 0.0
0.1 0.895 0.005 0.0
0.0 0.005 0.782 0.213
0.0 0.0 0.213 0.787


In [88]:
def camino(i,pasos,T):
    estado=[i]
    for k in range(pasos):
        estado.append(paso(estado[-1],T))
    return estado

In [89]:
viaje=camino(2,100,T)
viaje.count(0),viaje.count(1),viaje.count(2),viaje.count(3)

(0, 0, 50, 51)

In [38]:
"""Test to check wheter in long trips from random initial vertices we end up in a vertex with the prob
of the stationary distribution"""

m1=1000
m2=100

vertices=[0,1,2,3]
L=[[],[],[],[]]

for i in range(m1):
    a=random()
    if 0<a<=0.25:
        l=0
    elif 0.25<a<=0.5:
        l=1
    elif 0.5<a<=0.75:
        l=2
    elif 0.75<a<=1:
        l=3
    viaje=camino(l,m2,T)
    L[l].append(viaje[len(viaje)-1])
for v in vertex:
    print((L[v].count(0))/(len(L[v])),(L[v].count(1))/(len(L[v])),(L[v].count(2))/(len(L[v])),
          (L[v].count(3))/(len(L[v])))

0.3552123552123552 0.3590733590733591 0.14285714285714285 0.14285714285714285
0.27626459143968873 0.3463035019455253 0.20233463035019456 0.17509727626459143
0.17898832684824903 0.11673151750972763 0.3151750972762646 0.38910505836575876
0.13215859030837004 0.16299559471365638 0.3700440528634361 0.33480176211453744


In [386]:
P=np.matrix([[1/4,3/4],[1/5,4/5]])

"""Test to check the medium time we stay in each state when we run m1 paths of m2 steps"""

m1=1000
m2=100

vertices=[0,1]
L=[[],[]]

for i in range(m1):
    a=random()
    if 0<a<=0.5:
        l=0
    elif 0.5<a<=1:
        l=1
    viaje=camino(l,m2,P)
    for l in vertices:
        L[l].append(viaje.count(l))
for v in vertices:
    print((np.array(L[v])).sum())

21550
79450


In [397]:
T=np.matrix([[0.9,0.1,0,0],[0.1,0.895,0.005,0],[0,0.005,0.795,0.2],[0,0,0.2,0.8]])

"""Test to check the medium time we stay in each state when we run m1 paths of m2 steps"""

m1=1000
m2=100

vertices=[0,1,2,3]
L=[[],[],[],[]]

for i in range(m1):
    a=random()
    if 0<a<=0.25:
        l=0
    elif 0.25<a<=0.5:
        l=1
    elif 0.5<a<=0.75:
        l=2
    elif 0.75<a<=1:
        l=3
    viaje=camino(l,m2,T)
    for l in vertices:
        L[l].append(viaje.count(l))
for v in vertices:
    print((np.array(L[v])).sum())

25187
24169
26078
25566


In [48]:
D=np.array([[0,0,0,0,0,0,1,0,0,0],[0,0.3,0.3,0.1,0.3,0,0,0,0,0],[0,0,0.6,0,0,0,0,0.4,0,0],[0,0,0,1,0,0,0,0,0,0],
            [0,0.4,0,0,0,0.3,0,0.3,0,0],[0,0,0,0.9,0,0,0.1,0,0,0],[0,0,0,0,0,0,0,0,0,1],[0,0.8,0,0,0,0,0,0.2,0,0],
           [0,0,0,0,0,0,0,0,1,0],[1,0,0,0,0,0,0,0,0,0]])

In [49]:
D

array([[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.3,  0.3,  0.1,  0.3,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0.6,  0. ,  0. ,  0. ,  0. ,  0.4,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.4,  0. ,  0. ,  0. ,  0.3,  0. ,  0.3,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0.9,  0. ,  0. ,  0.1,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ],
       [ 0. ,  0.8,  0. ,  0. ,  0. ,  0. ,  0. ,  0.2,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ],
       [ 1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ]])